In [1]:
import nest_asyncio

nest_asyncio.apply()

from crewai import LLM
from dotenv import load_dotenv
import os

load_dotenv()

llm = LLM(
    model=os.getenv("FIREWORKS_MODEL_NAME"),
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY"),
)

In [2]:
from echo.runner import make_call
import asyncio
import nest_asyncio

nest_asyncio.apply()


test_clients = ["Shell", "Schneider electric"]


train_clients = [
    "ICICI bank",
    "Infosys",
    "University of Illinois",
    "Marks and spencer",
    "Mercedes Benz",
]

clients = test_clients + train_clients


inputs = {"call_type": "discovery", "seller": "Whatfix", "n_competitors": 3}

In [3]:
discovery_calls_data = asyncio.run(make_call("discovery", clients, inputs))

Making discovery call for ['Shell', 'Schneider electric', 'ICICI bank', 'Infosys', 'University of Illinois', 'Marks and spencer', 'Mercedes Benz']
Getting analysis Data


Getting Data:   0%|          | 0/7 [00:00<?, ?it/s]

Getting Data for Shell
Adding Analysis Data to Vector Store


Getting Data: 100%|██████████| 7/7 [00:00<00:00,  7.81it/s]

Splitting text into documents Discovery Analysis Buyer Data Fields Description: class BuyerDataExtracted(BaseModel):

	pain_points: List[str] # The pain points identified in the call.
	objections: List[str] # The objections identified in the call.
	time_lines: List[str] # The time lines identified in the call.
	success_indicators: List[str] # The success indicators identified in the call.
	budget_constraints: List[str] # The budget constraints identified in the call.
	competition: List[str] # The competitors identified in the call.
	decision_committee: List[str] # The members of the decision committee identified in the call.

# Pain_Points
   - maximizing technology investments
   - driving user adoption
   - balancing oil demand with climate goals
   - not seeing ROI on technology investments
   - users not adopting new tools quickly enough
   - impacting productivity and efficiency
# Objections
   - integration with existing systems
   - customization options
   - cost and implementa

In [4]:
from echo.step_templates.discovery import aget_simulation_data_for_client

print(inputs)
simulation_data = await aget_simulation_data_for_client(inputs, llm)

{'call_type': 'discovery', 'seller': 'Whatfix', 'n_competitors': 3}


KeyError: 'buyer'

In [ ]:
demo_calls_data = asyncio.run(make_call("demo", clients, inputs))

In [ ]:
pricing_call_data = asyncio.run(make_call("pricing", clients, inputs))

In [ ]:
negotiation_call_data = asyncio.run(make_call("negotiation", clients, inputs))

## Memory Module

## Using Memory

#### Memory Types

- Deal Summary

In [10]:
import enum
from typing import Dict
from llama_index.core.node_parser import SentenceSplitter
from echo.utils import db_storage_path
from llama_index.core import VectorStoreIndex
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition,
)

from llama_index.core.schema import TextNode
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb


CHUNK_SIZE = 8192
CHUNK_OVERLAP = 128
SIMILARITY_TOP_K = 3


class IndexType(enum.Enum):
    HISTORICAL = "historical"
    CURRENT_DEAL = "current_deal"
    BUYER_RESEARCH = "buyer_research"
    SELLER_RESEARCH = "seller_research"
    SALES_PLAYBOOK = "sales_playbook"


class CallType(enum.Enum):
    DISCOVERY = "discovery"
    DEMO = "demo"
    PRICING = "pricing"
    PROCUREMENT = "procurement"


def get_vector_index(index_name: str, index_type: IndexType):
    chroma_db_path = db_storage_path(index_name)
    db = chromadb.PersistentClient(path=str(chroma_db_path))
    chroma_collection = db.get_or_create_collection(f"{index_type.value}")
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    index = VectorStoreIndex.from_vector_store(vector_store)

    return index


def get_response(
    query: str,
    index: VectorStoreIndex,
    filters: Dict[str, str] | None = None,
    filter_operator: FilterOperator = FilterOperator.EQ,
    condition: FilterCondition = FilterCondition.AND,
):
    filters = filters or {}
    filters = MetadataFilters(
        filters=[
            MetadataFilter(key=k.lower(), value=v.lower(), operator=filter_operator)
            for k, v in filters.items()
        ],
        condition=condition,
    ).model_dump()

    return index.as_query_engine(filters=filters).query(query)


def get_nodes_from_documents(data: str, metadata: Dict[str, str]):
    splitter = SentenceSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    docs = splitter.split_text(data)
    return [TextNode(text=doc, metadata=metadata) for doc in docs]

## Queries

In [11]:
index_name = "Whatfix"
index_type = IndexType.HISTORICAL

chroma_db_path = db_storage_path(index_name)
db = chromadb.PersistentClient(path=str(chroma_db_path))
chroma_collection = db.get_or_create_collection(f"{index_type.value}")

In [12]:
index = get_vector_index(index_name, index_type)

In [ ]:
len(index._storage_context.vector_store._collection.get()["metadatas"])

In [14]:
def get_filters(metadata: Dict):
    filters = MetadataFilters(
        filters=[
            MetadataFilter(key=k, value=v["value"], operator=v["operator"])
            for k, v in metadata.items()
        ],
    )
    return filters

In [15]:
from IPython.display import Markdown, display


def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

In [ ]:
from llama_index.core.vector_stores import FilterOperator
metadata = {
    'call_type': {
        'value': 'discovery',
        'operator': FilterOperator.EQ
    },
    'seller': {
        'value': 'Whatfix',
        'operator': FilterOperator.EQ
    },
    'buyer': {
        'value': 'Shell',
        'operator': FilterOperator.NE
    },
    'company_size': {
        'value': 'Enterprise',
        'operator': FilterOperator.EQ
    },
}

filters = get_filters(metadata)
q = "What all pieces of pending information do i need to uncover from the buyer at the Discovery stage?"

sq1 , historical
sq2 - current index
sq3 - sq1,sq2,llm


sequence - subbqueries,index (historical, current, buyer, seller)


sq1 = "What all pieces of information about the buyer are uncovered in discovery calls of successful deals?"
qe = index.as_query_engine(filters=filters, similarity_top_k=SIMILARITY_TOP_K)
# display_prompt_dict(qe.get_prompts())
sq1_response = qe.query(sq1)
docs = index.as_retriever(filters=filters, similarity_top_k=SIMILARITY_TOP_K).retrieve(sq1)
docs


In [ ]:
sq1_response = qe.query(sq1)
print(".\n".join(str(sq1_response).split(".")))

In [ ]:
print(docs[0].text)

In [ ]:
# This is the cell we connect to flask

# given query and subquery flow as below, we will implement querying the respective indexes

# we use below filter when we want to query "current call index"
from llama_index.core.vector_stores import FilterOperator

metadata = {
    "call_type": {"value": "discovery", "operator": FilterOperator.EQ},
    "seller": {"value": "Whatfix", "operator": FilterOperator.EQ},
    "buyer": {"value": "Shell", "operator": FilterOperator.NE},
    "company_size": {"value": "Enterprise", "operator": FilterOperator.EQ},
}

filters = get_filters(metadata)


# query and subquery flow

# query = "What all pieces of pending information do i need to uncover from the buyer at the Discovery stage?"
# sub query 1 , historical
# sq1 = "What all pieces of information about the buyer are uncovered in discovery calls of successful deals?"
# sq1index = "historical"

# sub query 2 - current index
# sq2 = "What all pieces of information about the buyer are already uncovered in discovery calls for the curent deal?"
# sq2index = "current"


# sub query 3 - sq1,sq2,llm
# sq3 = """Given all the information we need to uncover from the buyer from sucessful discovery calls,
# and the information already uncovered till now for the current deal from the dicovery phase,
# extract what are the missing pieces of information that need to be unlocked in the upcoming discovery calls to make sure
# we have all the info that needs to be uncovered.
# Here is the info uncovered till now - sq2_response and here is all the info we need to uncover -sq1_response
# """
# sq3index = "llm"


# step 1 - create query engine for respective index
# historical index query engine
qe = index.as_query_engine(filters=filters, similarity_top_k=SIMILARITY_TOP_K)

# Step 2 - query the query engine with respective subquery
sq1_response = qe.query(sq1)


# do the same for other subquery / index pairs
# create query engine for current call index using filter
metadata2 = {
    "call_type": {"value": "discovery", "operator": FilterOperator.EQ},
    "seller": {"value": "Whatfix", "operator": FilterOperator.EQ},
    "buyer": {"value": "Shell", "operator": FilterOperator.EQ},
    "company_size": {"value": "Enterprise", "operator": FilterOperator.EQ},
}
filters2 = get_filters(metadata2)
qe = index.as_query_engine(filters=filters2, similarity_top_k=SIMILARITY_TOP_K)
sq2 = "What all pieces of information about the buyer are already uncovered in discovery calls for the curent deal?"
sq2_response = qe.query(sq2)

# do same for step 3 now
import os
from openai import OpenAI

sq3 = f"""Given all the information we need to uncover from the buyer from sucessful discovery calls,
and the information already uncovered till now for the current deal from the dicovery phase,
extract what are the missing pieces of information that need to be unlocked in the upcoming discovery calls to make sure 
we have all the info that needs to be uncovered.
Here is the info uncovered till now - {str(sq2_response)} and here is all the info we need to uncover - {str(sq1_response)}
"""

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """You are a sales assistant who given specific queries to resolve, need to generate responses to the queries using the queries and context provided.
            You need to help solve queries that help AE's prepare for calls using historical call info, current deals calls, and buyer and seller data
            """,
        },
        {
            "role": "user",
            "content": str(sq3),
        },
    ],
    model="gpt-4o",
)

print(chat_completion.choices[0].message.content)
